## Chapter 3: Finite Fields and Modular Arithmetic

### Computing the multiplicative inverse with Python

In [1]:
p = 17
print(pow(5, -1, p))
# 7
assert (5 * 7) % p == 1

7


Exercise: Use Python to compute the multiplicative inverse of 288 in the finite field of p = 311. You can check your work by validating that (288 * answer) % 311 == 1.

In [2]:
p = 311
mi = pow(288, -1, p)
assert (288 * mi) % p == 1

### The addition of multiplicative inverses is consistent with “regular” addition of fractions

In [3]:
p = 7
one_half = pow(2, -1, p)
one_third = pow(3, -1, p)
five_over_six = (pow(6, -1, p) * 5) % p

assert (one_half + one_third) % p == five_over_six

In [4]:
def compute_field_element_from_fraction(num, den, p):
    inv_den = pow(den, -1, p)
    return (num * inv_den) % p

### Finite field “division” does not suffer from precision loss

$$
x + y + z === 1 \\
x === y \\
y === z \\
$$

In [5]:
p = 11

# x, y, z have value 1/3
x = pow(3, -1, 11)
y = pow(3, -1, 11)
z = pow(3, -1, 11)

assert x == y;
assert y == z;
assert (x + y + z) % p == 1

### Finite field library in Python

In [6]:
import galois
GF7 = galois.GF(7) # GF7 is a class that wraps 7

one_half = GF7(1) / GF7(2)
one_third = GF7(1) / GF7(3)
five_over_six = GF7(5) / GF7(6)

assert one_half + one_third == five_over_six  

To calculate negative inverse

In [7]:
import galois
GF7 = galois.GF(7) # GF7 is a class that wraps 7

negative_two = -GF7(2)
assert negative_two + GF7(2) == 0

### The multiplication of fractions is also consistent

In [8]:

import galois
GF = galois.GF(11)

one_third = GF(1) / GF(3)
one_half = GF(1) / GF(2)
one_sixth = GF(1) / GF(6)

assert one_third * one_half == one_sixth


### Elements in a finite field do not need to be perfect squares to have a square root

In [9]:
numbers_with_roots = set()
p = 11
for i in range(0, p):
    numbers_with_roots.add(i * i % p)

print("numbers_with_roots:", numbers_with_roots)

numbers_with_roots: {0, 1, 3, 4, 5, 9}


### Computing the modular square root

In [10]:
from libnum import has_sqrtmod_prime_power, sqrtmod_prime_power
has_sqrtmod_prime_power(5, 11, 1) 
list(sqrtmod_prime_power(5, 11, 1))

[4, 7]

When `p` can be written as `4k+3` where `k` is an integer

In [ ]:
def mod_sqrt(x, p):
    assert (p - 3) % 4 == 0, "prime not 4k + 3"
    exponent = (p + 1) // 4
    return pow(x, exponent, p) # x ^ e % p

# Exercise: Use the code snippet above to compute the modular square root of 5 in the finite field of p = 23.
# The code will only give you one of the answers. How can you compute the other?

p = 23
x = 5
sqrt1 = mod_sqrt(x, p)
sqrt2 = p - sqrt1 # as the two roots are additive inverses of each other
print(sqrt1, sqrt2)

8 15


### Linear systems of equations in finite fields

$$
x + 2y=1\\
7x+3y=2
$$
Exercise: Write code to bruteforce every combination of (x, y) over x = 0..10, y = 0..10 to verify the above system has no solution over the finite field of p = 11.

In [9]:
import galois
GF11 = galois.GF(11)

for x in range(11):
    for j in range(11):
        expectOne =  GF11(x) + (2 * GF11(j))
        expectTwo =  (7 * GF11(x)) + (3 * GF11(j))
        if expectOne == 1 and expectTwo == 2:
            print(x, j) # found solution. but this should never hit as there is no solution

### Practice problems
In the problems below, use a finite field p of `21888242871839275222246405745257275088548364400416034343698204186575808495617`. Beware that the `galois` library takes a while to initialize a GF object, `galois.GF(p)`, of this size.

1. A dev creates an arithmetic circuit `x * y * z === 0` and `x + y + z === 0` with the intent of constraining all the signals to be zero. Find a counter example to this where the constraints are satisfied, but not all of x, y, and z are 0.

In [ ]:
import galois
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
GFBig = galois.GF(p)

for x in range(p):
    for y in range(p):
        for z in range(p):
            if GFBig(x) + GFBig(y) + GFBig(z) == 0 and GFBig(x) * GFBig(y) * GFBig(z) == 0: # circuit is satisfied
                if not(x == 0 and y == 0 and z == 0): # not the solution where all are 0
                    print(x, y, z)
                    break
        else:
            continue
        break
    else:
        continue
    break     
# 0 1 21888242871839275222246405745257275088548364400416034343698204186575808495616   

0 1 21888242871839275222246405745257275088548364400416034343698204186575808495616


2. A dev creates a circuit with the polynomial `x² + 2x + 3 === 11` and proves that 2 is a solution. What is the other solution? Hint: write the circuit as `x² + 2x - 8 === 0` then factor the polynomial by hand to find the roots. Finally, compute the congruent element of the roots in the finite field to find the other solution.

In [21]:
import galois
GFBig = galois.GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)
 
# using quadratic formula we know that the roots of the polynomial are 2 and -4
root1 = GFBig(2)
root2 = -GFBig(4) # This will calculate the congruent of -4
poly = galois.Poly([1,2,-8], GFBig)

assert poly(root1) == 0 # Ensuring that the roots satisfy the polynomial
assert poly(root2) == 0 # Ensuring that the roots satisfy the polynomial